# Importation des données
[To Do : continuer de décrire origine et format des données]

Les données sont localisées dans des fichiers XML, chaque fichier correspondant à une demande de brevets. Ces fichiers sont directement téléchargeables depuis le serveur FTP de l'INPI, accessible via FileZilla.

Pour rendre notre projet reproductible au maximum, bien que les données ne puissent pas être directement téléchargées par un utilisateur non autorisé par l'INPI, nous avons stocké les fichiers .zip tels que nous les avons initialement téléchargés dans le bucket diffusion de l'un de nous. 

Concernant leur format, ces fichiers compressés contiennent chacun les informations sur toutes les demandes de brevets déposés auprès de l'INPI chaque semaine. Il y a donc 52 dossiers par année, un pour chaque semaine. Dans ces dossiers, se trouvent des informations dont nous n'avons pas besoin (ex: des schémas d'invention). Nous n'utiliserons que les données présentes dans des fichiers XML, chacun de ces fichiers contenant de nombreuses informations sur une demande de brevet déposée dans la semaine concernée.

Le code suivant se charge de décompresser ces fichiers, d'identifier les fichiers XML au sein de leurs arborescences et d'en extraire toutes les informations.

In [ ]:
import os
import s3fs
import pandas as pd


from scripts.importation import process_all_years_s3

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [3]:
# NB : ce code tourne +7min
data_brevets = process_all_years_s3(fs, ROOT_S3_PATH)
data_brevets.head(2)

/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name


⚠️ Erreur XML dans mvallat/diffusion/projet/2022/FR_FRNEWST36_2022_25.zip/FR_FRNEWST36_2022_25/doc/3118340.xml: no element found: line 1, column 0


/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,...,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,...,NA,NA,NA,2017,NaN,NaN,NaN,NaN,NaN,NaN
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,...,NA,NA,NA,2017,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Exportation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
    data_brevets.to_parquet(file_out)

Notre base de données est maintenant stockée dans une bucket diffusion sur le ssp cloud, ce qui nous permet de ne pas avoir à repartir des fichiers compressés initiaux. 

**Pour importer directement les données sans passer par les étapes précédentes, on pourra utiliser le code suivant :**

In [15]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(10)

,doc-number,kind,country,status,publication_country,publication_doc-number,publication_date,publication_bopinum,publication_nature,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,applicant_1_postcode,applicant_1_country,applicant_2_orgname,applicant_2_address-1,applicant_2_city,applicant_2_postcode,applicant_2_country,applicant_3_orgname,applicant_3_address-1,applicant_3_city,applicant_3_postcode,applicant_3_country,inventor_1_last-name,inventor_1_first-name,inventor_1_address-1,inventor_1_city,inventor_1_postcode,inventor_1_country,inventor_2_last-name,inventor_2_first-name,inventor_2_address-1,inventor_2_city,inventor_2_postcode,inventor_2_country,inventor_3_last-name,inventor_3_first-name,inventor_3_address-1,inventor_3_city,inventor_3_postcode,inventor_3_country,agent_1_orgname,agent_1_address-1,agent_1_city,agent_1_postcode,agent_1_country,agent_2_orgname,agent_2_address-1,agent_2_city,agent_2_postcode,agent_2_country,agent_3_orgname,agent_3_address-1,agent_3_city,agent_3_postcode,agent_3_country,owner_1_last-name,owner_1_first-name,owner_1_address-1,owner_1_city,owner_1_postcode,owner_1_country,owner_2_last-name,owner_2_first-name,owner_2_address-1,owner_2_city,owner_2_postcode,owner_2_country,owner_3_last-name,owner_3_first-name,owner_3_address-1,owner_3_city,owner_3_postcode,owner_3_country,classification_1_text,classification_2_text,classification_3_text,abstract,citation_1_type,citation_1_text,citation_1_country,citation_1_doc-number,citation_1_date,citation_2_type,citation_2_text,citation_2_country,citation_2_doc-number,citation_2_date,citation_3_type,citation_3_text,citation_3_country,citation_3_doc-number,citation_3_date,year,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,3038352,A1,FR,PUBDEM,FR,3038352,20170106,2017-01,Brevet,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,FAURECIA INTERIEUR INDUSTRIE None,"2, RUE HENNAPE",NANTERRE,92000,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,BERTINELLI,RAPHAEL,20 RUE D'ECHAVANNE,CHENEBIER,70400,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,"2, PLACE D'ESTIENNE D'ORVES ,",PARIS,75009,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,F16B 5/07 20060101AFI20150702BHFR ...,F16B 5/08 20060101ALI20150702BHFR ...,B60J 5/00 20060101ALI20150702BHFR ...,La présente invention concerne un ensemble (10...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
1,3038278,A1,FR,PUBDEM,FR,3038278,20170106,2017-01,Brevet,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,IFP ENERGIES NOUVELLES None,1 ET 4 AVENUE DE BOIS-PREAU,RUEIL MALMAISON CEDEX,92852,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,MILOSAVLJEVIC,MISA,17 RUE CLAUDE BENARD,ERAGNY SUR OISE,95610,FR,ZITO,GIANLUCA,12 RUE DE PONTOISE,PARIS,75005,FR,NA,NA,NA,NA,NA,NA,NA,95 RUE D'AMSTERDAM,PARIS CEDEX 8,75378,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,B60W 20/00 20060101AFI20150630BHFR ...,B60W 10/08 20060101ALI20150630BHFR ...,B60W 10/115 20120101ALI20150630BHFR ...,Dans un véhicule hybride comprenant un moteur ...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2017,None,None,None,None,None,None
2,3038409,A1,FR,PUBDEM,FR,3038409,20170106,2017-01,Brevet,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE None,3 RUE MICHEL ANGE,PARIS,75016,FR,UNIVERSITE D'AIX-MARSEILLE None,58 BOULEVARD CHARLES LIVON,MARSEILLE CEDEX 7,13284,FR,ECOLE CENTRALE DE MARSEILLE None,TECHNOPOLE DE CHATEAU GOMBERT\n38 RUE FREDERIC...,MARSEILLE,13013,FR,DUPONT,GUILLAUME,28 RUE DE VERDUN,MARSEILLE,13005,FR,ENOCH,STEFAN,15 RUE DE L'ESCALET\nLA COLLINE N°17,MARSEILLE,13013,FR,MOLIN,BERNARD,20 CHEMIN DE LA RESQUILLETTE,VENELLES,13770,FR,NA,310 AVENUE BERTHELOT,LYON,69008,FR,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,G06F 17/50 20060101AFI20150701BHFR ...,G06F 17/11 20060101ALI20150701BHFR ...,E02B 3/06 20060

# Nettoyage donnée et focus
Liste des colones et identification des colonnes à supprimer


In [ ]:
pd.set_option('display.max_columns', None)


Index(['doc-number', 'kind', 'country', 'status', 'publication_country',
       'publication_doc-number', 'publication_date', 'publication_bopinum',
       'publication_nature', 'invention-title',
       ...
       'citation_3_country', 'citation_3_doc-number', 'citation_3_date',
       'year', 'application_country', 'application_doc-number',
       'application_date', 'last-fee-payement', 'next-fee-payement',
       'date-search-completed'],
      dtype='object', length=102)

On conserve donc les colonnes date, invention_title, classification_1_text qui correspond à la classification internationale et l'abstract

In [17]:
data_brevets_clean = data_brevets[['publication_date','invention-title','classification_1_text','abstract']]
data_brevets_clean.head(5)

,publication_date,invention-title,classification_1_text,abstract
0,20170106,ENSEMBLE POUR PANNEAU DE PORTE ET PANNEAU ASSOCIE,F16B 5/07 20060101AFI20150702BHFR ...,La présente invention concerne un ensemble (10...
1,20170106,VEHICULE HYBRIDE COMPRENANT UNE SUPPRESSION DE...,B60W 20/00 20060101AFI20150630BHFR ...,Dans un véhicule hybride comprenant un moteur ...
2,20170106,PROCEDE DE CONCEPTION ASSISTEE PAR ORDINATEUR ...,G06F 17/50 20060101AFI20150701BHFR ...,Ce procédé de conception assistée par ordinate...
3,20170106,DOCUMENT ELECTRONIQUE TEL QU'UNE CARTE A PUCE ...,G06K 19/077 20060101AFI20150630BHFR ...,La fabrication d'un document électronique comp...
4,20170106,PROCEDE DE SYNTHESE EN CONTINU D'UN ELASTOMERE...,C08C 19/44 20060101AFI20150702BHFR ...,L'invention concerne un procédé de synthèse en...


première stat desc

In [ ]:
#CA MARCHE PAS pip install -r requirements.txt 
import matplotlib.pyplot as plt
# Extraire la section (première lettre) et la classe (lettre + 2 chiffres)
data_brevets_clean['section'] = data_brevets_clean['classification_1_text'].str[0]
data_brevets_clean['classe'] = data_brevets_clean['classification_1_text'].str[:3]

# date en datetime et extraction de l'année
data_brevets_clean['publication_date'] = pd.to_datetime(data_brevets_clean['publication_date'], format='%Y%m%d')
data_brevets_clean['annee'] = data_brevets_clean['publication_date'].dt.year

#  ANALYSE PAR SECTION (lettre seule)
print("ANALYSE PAR SECTION IPC (A, B, C, D, E, F, G, H)")

# Comptage par section
comptage_section = data_brevets_clean['section'].value_counts().sort_index()

# Statistiques avec pourcentages
df_stats_section = pd.DataFrame({
    'nombre': comptage_section,
    'pourcentage': (comptage_section / comptage_section.sum() * 100).round(2)
})
print("\n", df_stats_section)

# Visualisation - Distribution par section
fig, axes = plt.subplots(1, 2, figsize=(15, 5))


# Graphique en camembert
axes[1].pie(comptage_section, labels=comptage_section.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Répartition en pourcentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Évolution temporelle par section
evolution_section = data_brevets_clean.groupby(['annee', 'section']).size().unstack(fill_value=0)
print("\n", "="*60)
print("ÉVOLUTION PAR SECTION ET PAR ANNÉE")
print("="*60)
print(evolution_section)

plt.figure(figsize=(14, 7))
evolution_section.plot(kind='line', marker='o', linewidth=2)
plt.title('Évolution du nombre de brevets par section IPC', fontsize=14, fontweight='bold')
plt.xlabel('Année', fontsize=12)
plt.ylabel('Nombre de brevets', fontsize=12)
plt.legend(title='Section IPC', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ========== ANALYSE PAR CLASSE (G01, G06, etc.) ==========
print("ANALYSE PAR CLASSE IPC (ex: G01, G06, B60, etc.)")

# Comptage par classe
comptage_classe = data_brevets_clean['classe'].value_counts().sort_index()

# Top 20 des classes les plus fréquentes
top20_classes = comptage_classe.head(20)

# Statistiques avec pourcentages
df_stats_classe = pd.DataFrame({
    'nombre': top20_classes,
    'pourcentage': (top20_classes / comptage_classe.sum() * 100).round(2)
})
print("\nTop 20 des classes les plus fréquentes:")
print(df_stats_classe)

# Visualisation - Top 20 des classes
plt.figure(figsize=(14, 7))
top20_classes.plot(kind='barh', color='coral')
plt.title('Top 20 des classes IPC les plus fréquentes', fontsize=14, fontweight='bold')
plt.xlabel('Nombre de brevets', fontsize=12)
plt.ylabel('Classe IPC', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# Évolution temporelle des principales classes
top10_classes_names = comptage_classe.head(10).index
evolution_classe = data_brevets_clean[data_brevets_clean['classe'].isin(top10_classes_names)].groupby(['annee', 'classe']).size().unstack(fill_value=0)

print("\n", "="*60)
print("ÉVOLUTION DES TOP 10 CLASSES PAR ANNÉE")
print("="*60)
print(evolution_classe)

plt.figure(figsize=(14, 7))
evolution_classe.plot(kind='line', marker='o', linewidth=2)
plt.title('Évolution du nombre de brevets - Top 10 des classes IPC', fontsize=14, fontweight='bold')
plt.xlabel('Année', fontsize=12)
plt.ylabel('Nombre de brevets', fontsize=12)
plt.legend(title='Classe IPC', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Résumé final
print(f"Nombre total de brevets: {len(data_brevets_clean)}")
print(f"Nombre de sections différentes: {data_brevets_clean['section'].nunique()}")
print(f"Nombre de classes différentes: {data_brevets_clean['classe'].nunique()}")
print(f"Période couverte: {data_brevets_clean['annee'].min()} - {data_brevets_clean['annee'].max()}")


SyntaxError: invalid syntax (4141711738.py, line 1)